In [1]:
# Code for hyperparameter optimization using Bayesian methods from
# hyperopt library


#Setting GPU to use for hyperparameter optimization


import os
import tensorflow
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152

# 0 is 1080ti and 1 is 2080ti 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/mikedoho/PycharmProjects/OPO_LN_Pred_Uncertainty/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mikedoho/PycharmProjects/OPO_LN_Pred_Uncertainty/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mikedoho/PycharmProjects/OPO_LN_Pred_Uncertainty/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtyp

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13200412843472123930
]


In [2]:
# importing libraries for keras and tensorflow along with other general libraries

import os
import scipy.io
import h5py
import shutil
from colorama import Fore, Style, Back
from scipy import interp
from itertools import cycle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_fscore_support, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import re
from tqdm import tqdm_notebook as tqdm

import keras
import tensorflow as tf

from keras import layers
from keras import regularizers
from keras.engine import Input, Model
from keras.models import Sequential, load_model
from keras.layers import UpSampling3D, PReLU, Deconvolution3D
from keras.layers import Activation, Dense, Input, BatchNormalization
from keras.layers import Conv3D, concatenate, MaxPooling3D, Flatten, Dropout
from keras.layers import AveragePooling3D, GlobalAveragePooling3D, GlobalMaxPooling3D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils, plot_model

# Libraries for hyperopt

from hyperopt import Trials, STATUS_OK, STATUS_FAIL, tpe, fmin, space_eval, pyll, hp
import pickle
import traceback

from keras import backend as K
K.set_image_data_format("channels_last")

try:
    from keras.engine import merge
except ImportError:
    from keras.layers.merge import concatenate


# prevents memory allocation errors when using GPU
from keras import backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

print('Versions of libraries')
print('numpy version: ', np.version.version)
print('matplotlib version: ', matplotlib.__version__)
print('sklearn version: ', sklearn.__version__)
print('keras version: ', keras.__version__)
print('tensorflow version: ', tf.VERSION)


Using TensorFlow backend.


Versions of libraries
numpy version:  1.18.2
matplotlib version:  3.2.1
sklearn version:  0.22.2.post1
keras version:  2.2.4
tensorflow version:  1.9.0


In [3]:
display('Current settings; dont delete')

train_dataset = [1,3,4]
valid_dataset = [2]
test_dataset = [5]

'Current settings; dont delete'

In [4]:
main_dir = '/home/mikedoho/Desktop/utsw_hn_ai/opo_hn_LN_prediction'
model_dir = os.path.join(main_dir, 'Models')

data_dir = '/media/hdd_1_2TB/Data/OPSCC_Data/train_val_test_split/'

#training data directory
data_train_dir = os.path.join(data_dir, 'Train')
data_train_ct_dir = os.path.join(data_train_dir, 'CT_Train')
data_train_pet_dir = os.path.join(data_train_dir, 'PET_Train')
data_train_label_dir = os.path.join(data_train_dir, 'Label_Train')

# validation data directory
data_val_dir = os.path.join(data_dir, 'Validation')
data_val_ct_dir = os.path.join(data_val_dir, 'CT_Val')
data_val_pet_dir = os.path.join(data_val_dir, 'PET_Val')
data_val_label_dir = os.path.join(data_val_dir, 'Label_Val')

# Test data directory
data_test_dir = os.path.join(data_dir, 'Test')
data_test_ct_dir = os.path.join(data_test_dir, 'CT_Test')
data_test_pet_dir = os.path.join(data_test_dir, 'PET_Test')
data_test_label_dir = os.path.join(data_test_dir, 'Label_Test')


dir_dict = {
    'main_dir': main_dir, 
    'dir_data': data_dir,
    
    'dir_train_ct': data_train_ct_dir, 
    'dir_train_pet': data_train_pet_dir,
    'dir_train_label': data_train_label_dir,

    'dir_val_ct': data_val_ct_dir, 
    'dir_val_pet': data_val_pet_dir,
    'dir_val_label': data_val_label_dir,
    
    'dir_test_ct': data_test_ct_dir, 
    'dir_test_pet': data_test_pet_dir,
    'dir_test_label': data_test_label_dir,
    
    'dir_model': model_dir
}

In [5]:
def load_train_data_batch_generator(batch_size=32, rows_in=72, cols_in=72, zs_in=48, 
                                    channels_in=2, num_classes=2, 
                                    dir_dict=dir_dict):
    '''
    Training data generator
    batch_size: batch size
    rows_in: # rows (pixels) in image
    cols_in: # cols (pixels) in image
    zs_in: # images in CT/PET
    channels_in: # of image modalities (CT and PET)
    num_classes: # of classes that model is predicting
    dir_dict: dictionary containing directories 
    '''  
       
    # required when using hyperopt
    batch_size = int(batch_size)
    # if not: TypeError: 'float' object cannot be interpreted as an integer
    
    # names of all files in directory
    fnames = os.listdir(dir_dict['dir_train_ct'])
    
    y_train = np.zeros((batch_size, num_classes))
    x_train = np.zeros((batch_size, rows_in, cols_in, zs_in, channels_in))

    while True:
        count = 0
        for fname in np.random.choice(fnames, batch_size, replace=False):

            data_label = scipy.io.loadmat(os.path.join(dir_dict['dir_train_label'], fname), 
                                          verify_compressed_data_integrity=False)['label']
            
#             data_label = data_label[0]

            # changing one hot encoding to integer
            integer_label = np.argmax(data_label[0], axis=0)
            y_train[count,:] = data_label
#             print(fname)
            

            # Loading train ct w/ c and pet/ct combo 
            train_ct = scipy.io.loadmat(os.path.join(dir_dict['dir_train_ct'], fname),
                                        verify_compressed_data_integrity=False)['roi_patch_ct']
            train_pet = scipy.io.loadmat(os.path.join(dir_dict['dir_train_pet'], fname), 
                                         verify_compressed_data_integrity=False)['roi_patch_pet']
            
            x_train[..., 0] = train_ct/1
            x_train[..., 1] = train_pet/14.30099

            count += 1

        yield(x_train, y_train)


def load_val_data_batch_generator(batch_size=32, rows_in=72, cols_in=72, zs_in=48, 
                                  channels_in=2, num_classes=2, 
                                  dir_dict=dir_dict):
    '''
    Valdiation data generator
    batch_size: batch size
    rows_in: # rows (pixels) in image
    cols_in: # cols (pixels) in image
    zs_in: # images in CT/PET
    channels_in: # of image modalities (CT and PET)
    num_classes: # of classes that model is predicting
    dir_dict: dictionary containing directories 
    '''  
    
    # required when using hyperopt
    batch_size = int(batch_size)
    # if not: TypeError: 'float' object cannot be interpreted as an integer

    # names of all files in directory 
    fnames = os.listdir(dir_dict['dir_val_ct'])
    
    y_val = np.zeros((batch_size, num_classes))
    x_val = np.zeros((batch_size, rows_in, cols_in, zs_in, channels_in))

    while True:
        count = 0
        for fname in np.random.choice(fnames, batch_size, replace=False):

            data_label = scipy.io.loadmat(os.path.join(dir_dict['dir_val_label'], fname), 
                                          verify_compressed_data_integrity=False)['label']

            # changing one hot encoding to integer
            integer_label = np.argmax(data_label[0], axis=0)
            y_val[count,:] = data_label

            # Loading val ct w/ c and pet/ct combo 
            val_ct = scipy.io.loadmat(os.path.join(dir_dict['dir_val_ct'], fname), 
                                      verify_compressed_data_integrity=False)['roi_patch_ct']
            val_pet = scipy.io.loadmat(os.path.join(dir_dict['dir_val_pet'], fname), 
                                       verify_compressed_data_integrity=False)['roi_patch_pet']
            
            x_val[..., 0] = val_ct/1
            x_val[..., 1] = val_pet/14.30099

            count += 1
                       
        yield(x_val, y_val)

In [6]:
def load_train_data_batch(batch_size=32, rows_in=72, cols_in=72, zs_in=48, 
                          channels_in=2, num_classes=2, 
                          dir_dict=dir_dict):
    '''
    Training data loader for AUC curve
    batch_size: batch size
    rows_in: # rows (pixels) in image
    cols_in: # cols (pixels) in image
    zs_in: # images in CT/PET
    channels_in: # of image modalities (CT and PET)
    num_classes: # of classes that model is predicting
    dir_dict: dictionary containing directories 
    '''  
    
    # required when using hyperopt
    batch_size = int(batch_size)
    # if not: TypeError: 'float' object cannot be interpreted as an integer
    
    fnames = os.listdir(dir_dict['dir_train_ct'])
    
    y_train = np.zeros((batch_size, num_classes))
    x_train = np.zeros((batch_size, rows_in, cols_in, zs_in, channels_in))

    while True:
        count = 0
        for fname in np.random.choice(fnames, batch_size, replace=False):

            data_label = scipy.io.loadmat(os.path.join(dir_dict['dir_train_label'], fname), 
                                          verify_compressed_data_integrity=False)['label']

            # changing one hot encoding to integer
            integer_label = np.argmax(data_label[0], axis=0)
            y_train[count,:] = data_label

            # Loading train ct w/ c and pet/ct combo 
            train_ct = scipy.io.loadmat(os.path.join(dir_dict['dir_train_ct'], fname),
                                        verify_compressed_data_integrity=False)['roi_patch_ct']
            train_pet = scipy.io.loadmat(os.path.join(dir_dict['dir_train_pet'], fname), 
                                         verify_compressed_data_integrity=False)['roi_patch_pet']
            
            x_train[..., 0] = train_ct/1
            x_train[..., 1] = train_pet/14.30099

            count += 1

        return(x_train, y_train)


def load_val_data_batch(batch_size=32, rows_in=72, cols_in=72, zs_in=48, 
                        channels_in=2, num_classes=2, 
                        dir_dict=dir_dict):
    '''
    Validation data loader for AUC curve
    batch_size: batch size
    rows_in: # rows (pixels) in image
    cols_in: # cols (pixels) in image
    zs_in: # images in CT/PET
    channels_in: # of image modalities (CT and PET)
    num_classes: # of classes that model is predicting
    dir_dict: dictionary containing directories 
    '''  
    
    # required when using hyperopt
    batch_size = int(batch_size)
    # if not: TypeError: 'float' object cannot be interpreted as an integer

    fnames = os.listdir(dir_dict['dir_val_ct'])
    
    y_val = np.zeros((batch_size, num_classes))
    x_val = np.zeros((batch_size, rows_in, cols_in, zs_in, channels_in))

    while True:
        count = 0
        for fname in np.random.choice(fnames, batch_size, replace=False):

            data_label = scipy.io.loadmat(os.path.join(dir_dict['dir_val_label'], fname), 
                                          verify_compressed_data_integrity=False)['label']

            # changing one hot encoding to integer
            integer_label = np.argmax(data_label[0], axis=0)
            y_val[count,:] = data_label

            # Loading val ct w/ c and pet/ct combo 
            val_ct = scipy.io.loadmat(os.path.join(dir_dict['dir_val_ct'], fname), 
                                      verify_compressed_data_integrity=False )['roi_patch_ct']
            val_pet = scipy.io.loadmat(os.path.join(dir_dict['dir_val_pet'], fname),
                                       verify_compressed_data_integrity=False)['roi_patch_pet']
            
            x_val[..., 0] = val_ct/1
            x_val[..., 1] = val_pet/14.30099

            count += 1

        return(x_val, y_val)
    
def load_test_data_batch(batch_size=32, rows_in=72, cols_in=72, zs_in=48, 
                        channels_in=2, num_classes=2, 
                        dir_dict=dir_dict):
    '''
    Validation data loader for AUC curve
    batch_size: batch size
    rows_in: # rows (pixels) in image
    cols_in: # cols (pixels) in image
    zs_in: # images in CT/PET
    channels_in: # of image modalities (CT and PET)
    num_classes: # of classes that model is predicting
    dir_dict: dictionary containing directories 
    '''  
    
    # required when using hyperopt
    batch_size = int(batch_size)
    # if not: TypeError: 'float' object cannot be interpreted as an integer

    fnames = os.listdir(dir_dict['dir_test_ct'])
    
    y_test = np.zeros((batch_size, num_classes))
    x_test = np.zeros((batch_size, rows_in, cols_in, zs_in, channels_in))

    while True:
        count = 0
        for fname in np.random.choice(fnames, batch_size, replace=False):

            data_label = scipy.io.loadmat(os.path.join(dir_dict['dir_test_label'], fname),
                                          verify_compressed_data_integrity=False)['label']

            # changing one hot encoding to integer
            integer_label = np.argmax(data_label[0], axis=0)
            y_test[count,:] = data_label

            # Loading val ct w/ c and pet/ct combo 
            val_ct = scipy.io.loadmat(os.path.join(dir_dict['dir_test_ct'], fname), 
                                      verify_compressed_data_integrity=False)['roi_patch_ct']
            val_pet = scipy.io.loadmat(os.path.join(dir_dict['dir_test_pet'], fname), 
                                       verify_compressed_data_integrity=False)['roi_patch_pet']
            
            x_test[..., 0] = val_ct/1
            x_test[..., 1] = val_pet/14.30099

            count += 1

        return(x_test, y_test)

In [7]:
'''
Hyper parameter space for UNET model 
'''

unet_hype_space = {
    
    'nb_classes': 2,
    'input_shape': np.shape(np.zeros((72, 72, 48, 2))),
    'batch_size': hp.quniform('batch_size', 10, 20, 2),
    'epochs': hp.choice('epochs_mike', [50]),
    'class_weights': hp.choice('class_weights', [1, 2, 3]),
    
    'learning_rate': hp.choice('learning_rate', [1e-5, 5*1e-5, 1e-4]),
    
    'conv_dropout_drop_proba': hp.choice('conv_dropout_proba', [0.01, 0.015]),   
    'fc_dropout_drop_proba': hp.choice('fc_dropout_proba', [0.1, 0.2]),
    
    'final_activation': hp.choice('final_activation',["softmax"]),
    'metrics': hp.choice('metrics', ['accuracy']),
    'loss': hp.choice('loss',['categorical_crossentropy']),
    
    # Use batch normalisation at more places?
    'use_BN': hp.choice('use_BN', [False, True]),
    'use_DO_conv': hp.choice('use_DO_conv', [True]),
    'use_DO_dense': hp.choice('use_DO_dense', [True]),
            
    'MP_stride': hp.choice('MP_stride_unet', [(2,2,2), (3,3,3)]),
    'MP_pool': hp.choice('MP_pool_unet', [(3,3,3), (2,2,2)]),
      
    # UNET specific
    'UNET_depth': hp.choice('UNET_depth', [1, 2, 3]),
    'kernel_size': hp.choice('kernel_size', [(2,2,2), (3,3,3)]),
    'padding': hp.choice('padding', ['valid']),
    'n_filters': hp.choice('n_filters', [20, 30, 15]), 
    'use_REG': hp.choice('use_REG', [False, True]),
    'l2_reg': hp.loguniform('l2_reg', -3, 3),
#     'l1_reg': hp.loguniform('l1_reg', -2, 2),
    'instance_normalization': hp.choice('instance_normalization', [False, True]),
    'deconvolution': hp.choice('deconvolution', [True]),
    'n_first_dense_filters': hp.choice('n_first_dense_filters', [10, 20, 40, 60]),
    'conv_activation': hp.choice('conv_activation', ['relu']),
    
    'which_model': 'unet'
    
}

In [8]:
def unet_model_3d(space):
    """
    Builds the 3D UNet Keras model.f
    :param metrics: List metrics to be calculated during model training.
    :param n_filters: The number of filters that the first layer in the convolution network will have. Following
    layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
    to train the model.
    :param UNET_depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
    layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
    :param input_shape: Shape of the input data (x_size, y_size, z_size, n_channels). The x, y, and z sizes must be
    divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth.
    :param pool_size: Pool size for the max pooling operations.
    :param num_classes: Number of binary labels that the model is learning.
    :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
    increases the amount memory required during training.
    :return: Untrained 3D UNet Model
    """
    
    print('Creating Unet...')
    print('Filter size output is truncated')
    
    inputs = Input(space['input_shape'])
    current_layer = inputs
    levels = list()

    # add levels with max pooling
    for layer_depth in range(space['UNET_depth']):
        layer1 = create_convolution_block(space=space,
                                          input_layer=current_layer, 
                                          n_filters=space['n_filters']*(2**layer_depth))
        
        layer2 = create_convolution_block(space=space,
                                          input_layer=layer1, 
                                          n_filters=space['n_filters']*(2**layer_depth)*2)
        
        if layer_depth < space['UNET_depth'] - 1:
            current_layer = MaxPooling3D(pool_size=space['MP_pool'])(layer2)
            levels.append([layer1, layer2, current_layer])
        else:
            current_layer = layer2
            levels.append([layer1, layer2])

    # add levels with up-convolution or up-sampling
    for layer_depth in range(space['UNET_depth']-2, -1, -1):
        up_convolution = get_up_convolution(pool_size=space['MP_pool'], 
                                            deconvolution=space['deconvolution'],
                                            n_filters=current_layer._keras_shape[-1])(current_layer)
     
        concat = concatenate([up_convolution, levels[layer_depth][1]], axis=-1)
        
        current_layer = create_convolution_block(space=space,
                                                 n_filters=levels[layer_depth][1]._keras_shape[-1],
                                                 input_layer=concat)
        
        current_layer = create_convolution_block(space=space,
                                                 n_filters=levels[layer_depth][1]._keras_shape[-1],
                                                 input_layer=current_layer)

    final_convolution = Conv3D(space['nb_classes'], (1, 1, 1))(current_layer)
    act = Activation(space['conv_activation'])(final_convolution)
    
    flatten = Flatten()(act)
    dense1 = Dense(space['n_first_dense_filters'], activation='relu')(flatten)
    
    if space['use_DO_dense']:
        dense1 = Dropout(space['fc_dropout_drop_proba'])(dense1)
    
    dense2 = Dense(space['nb_classes'], 
                   activation=space['final_activation'], name='predictions')(dense1)
    
    #print('Proof of filter size: {}'.format(current_layer._keras_shape[-1]))
    
    model = Model(inputs=inputs, outputs=dense2)
    
    model.compile(optimizer=Adam(lr=space['learning_rate']*space['lr_rate_mult']),
                 loss=space['loss'],
                 metrics=[space['metrics']])
    
    
#     print('Plotting model...')
#     plot_model(model, to_file='unet_model.png')
#     print('Picture made. Please see in ', os.getcwd())
#     print('\nCurrent parameter space \n',space)
#     print('\n')
    
    return model

def create_convolution_block(space,
                             input_layer, 
                             n_filters):
    """
    Used in creating UNET
    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    
    if space['use_REG']:
        layer = Conv3D(filters=space['n_filters'], 
                       kernel_size=space['kernel_size'], 
                       padding=space['padding'], 
                       strides=space['MP_stride'],
                       kernel_regularizer=regularizers.l2(space['l2_reg']))(input_layer)
        
    else: layer = Conv3D(filters=space['n_filters'], 
                         kernel_size=space['kernel_size'], 
                         padding=space['padding'], 
                         strides=space['MP_stride'])(input_layer)
    
    
    if space['use_BN']:
        layer = BatchNormalization(axis=-1)(layer)
        
    if space['use_DO_conv']:
        layer = Dropout(space['conv_dropout_drop_proba'])(layer)    
    
    act = Activation(space['conv_activation'])(layer)
    
    return act

def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    ''' Used in creating UNET'''
    
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)

In [9]:
def roc_auc_plot(y_true, y_pred, n_classes=2, lw=2, data_title='Unspecified'):
    '''
    plot ROC and compute auc using sklearn. Taken from 
    https://github.com/Tony607/ROC-Keras/blob/master/ROC-Keras.ipynb.
    #y_test: actual results
    #y_score: predicted results
    #n_classes: number of classes
    #lw: linewidth
    '''

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(1)
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC--{data_title}')
    plt.legend(loc="lower right")
    plt.show()
    
    
def dr_friendly_measures(y_true, y_pred):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    
    name1 = ['specificity', 'sensitivity', 'ppv', 'npv']
    name2 = [specificity, sensitivity, ppv, npv]
    
    for i in range(len(name1)):
        print(f"{name1[i]}: {np.round(name2[i], 3)}")
    
    return specificity, sensitivity, ppv, npv

In [12]:
def train_model(space,
                dir_dict=dir_dict, 
                plot_vis=True, 
                auroc=True,
                data_vis=False,
                gen_compare=False,
                verbose=1):
    '''
    Trains the model
    Callbacks include EarlyStopping and ReduceLROnPlateau
    ROC curves and precision, recall, fscore calculate for validation and test data
    Current model is compared to saved model then saves current model if better
    Calculates specified value for hyperopt minimization
    '''
        
    model = unet_model_3d(space)
   
    
    call_back = [EarlyStopping(monitor='val_loss', patience=40, min_delta=0.005,
                               restore_best_weights=True, verbose=verbose),
                 ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                              patience=20, min_lr=5e-8,  
                              verbose=verbose)]
    
    print('Evaluating parameter space\n')
    
    display(round(len(os.listdir(dir_dict['dir_val_label']))/space['batch_size']))
    display(len(os.listdir(dir_dict['dir_val_label'])))
    display(space['batch_size'])
    
    # History
    history = model.fit_generator(
            generator=load_train_data_batch_generator(batch_size=space['batch_size']),
            steps_per_epoch=round(len(os.listdir(dir_dict['dir_train_ct']))/space['batch_size']),
            epochs=space['epochs'],
            verbose=verbose,
            callbacks=call_back,
            validation_data=load_val_data_batch_generator(batch_size=space['batch_size']),
            validation_steps=round(len(os.listdir(dir_dict['dir_val_ct']))/space['batch_size']),
            class_weight={0:1, 1:3}
    ).history
    
    
    # Test model to obtain score/acc:
    
    
    score = model.evaluate_generator(generator = load_val_data_batch_generator(batch_size=space['batch_size']), 
                                     steps = round(len(os.listdir(dir_dict['dir_val_label']))/space['batch_size']))
    
        
    if auroc:
        # Creating ROC curve for validation data
        val_x_auc, val_y_auc =  load_val_data_batch(len(os.listdir(dir_dict['dir_val_label'])))
        y_pred = model.predict(x=val_x_auc)
        roc_auc_plot(val_y_auc, y_pred, data_title='VALIDATION')
        
        
        target_names = ['class '+str(x) for x in range(space['nb_classes'])]
        
        print('VALIDATION INFORMATION')
        
        # printing precision, recall, and fscore for each class for validation data
        print('\n', classification_report(np.argmax(val_y_auc, axis=-1), 
                                          np.argmax(y_pred, axis=-1), 
                                          target_names=target_names))     
        
        
        def weight_avg(x, y):
            '''Compute weighted average'''
            numarator = []
            numarator = [x[i]*y[i]for i in range(len(x))]
            num_sum = sum(numarator)
            weight_avg = num_sum/sum(y)

            return weight_avg
    
        
        precision, recall, fscore, support = precision_recall_fscore_support(np.argmax(val_y_auc, axis=-1), 
                                                                                 np.argmax(y_pred, axis=-1))

        specificity, sensitivity, ppv, npv = dr_friendly_measures(np.argmax(val_y_auc, axis=-1), 
                                                                      np.argmax(y_pred, axis=-1))
        
        # being used as metrics for optimization in hyperopt *min(-metric)
        avg_fscore = weight_avg(fscore, support)
        avg_fscore_metric = weight_avg(fscore, support)
        
        # focus scores to be disproportionally  worse or better
        if sensitivity == 0 or specificity == 0:
            hyper_opt_score = 1
        elif sensitivity >= 0.85 and specificity >=0.85 and sensitivity > specificity:
            hyper_opt_score = 4.5
        elif sensitivity >= 0.85 and specificity >=0.85: 
            hyper_opt_score = 4.3
        elif sensitivity >= 0.8 and specificity >=0.8 and sensitivity > specificity:
            hyper_opt_score = 4.2
        elif sensitivity >= 0.8 and specificity >=0.8:
            hyper_opt_score = 4
        else:
            hyper_opt_score = (3*sensitivity + specificity)
        
        eval_val_metric = [precision, recall, fscore, avg_fscore]
        eval_val_metric_dr = [specificity, sensitivity, ppv, npv]
               

        # Process repeated above but to see test data 
        test_x_auc, test_y_auc =  load_test_data_batch(len(os.listdir(dir_dict['dir_test_label'])))
        y_pred_test = model.predict(x=test_x_auc)
        roc_auc_plot(test_y_auc, y_pred_test, data_title='TEST')
        
        
        print('TEST INFORMATION')
        
        # printing precision, recall, and fscore for each class for test data
        print('\n', classification_report(np.argmax(test_y_auc, axis=-1), 
                                          np.argmax(y_pred_test, axis=-1), 
                                          target_names=target_names))
        
        precision, recall, fscore, support = precision_recall_fscore_support(np.argmax(test_y_auc, axis=-1), 
                                                                             np.argmax(y_pred_test, axis=-1))
        
        dr_friendly_measures(np.argmax(test_y_auc, axis=-1), 
                                       np.argmax(y_pred_test, axis=-1))
    
    max_acc = max(history['val_acc'])
    
    if data_vis:
        # obtain numeric representation of model evaluation
        print('\n')
        print(history.keys())
        print(history)
        print(model.metrics_names)
        print(score)
        print('\n\n')
    
    if plot_vis:
        # Plot training & validation accuracy values
        plt.plot(history['val_acc'])
        plt.plot(history['acc'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['val_acc', 
                    'acc'], loc='upper left')
        plt.show()
        
        try:
            history['val_sk_auroc']
        except KeyError:
            print('no val_sk_auroc')
        else:
            #Plot training & validation auc/sk_auc
            plt.plot(history['val_sk_auroc'])
            plt.plot(history['sk_auroc'])
    #         plt.plot(history['val_auc'])
    #         plt.plot(history['auc'])
            plt.title('Model AUC/sk_AUC')
            plt.ylabel('AUC')
            plt.xlabel('Epoch')
            plt.legend(['val_sk_auc', 'sk_auc'], loc='upper left')
            plt.show()

        # Plot training & validation loss values
        plt.plot(history['val_loss'])
        plt.plot(history['loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['val_loss', 'loss'], loc='upper left')
        plt.show()
    
    results = {
        # We plug "-hyper_opt_score" as a
        # minimizing metric named 'loss' by Hyperopt.
        'loss': -hyper_opt_score,
        'real_loss': score[0],
        # stats:
        'best_loss': min(history['val_loss']),
        'best_accuracy': max(history['val_acc']),
        'end_loss': score[0],
        'end_accuracy': score[1],
        # Misc:
        'space': space,
        'history': history,
        'status': STATUS_OK
    }
    

    return model, results, eval_val_metric, eval_val_metric_dr, space

In [13]:
#LIYUAN THIS IS THE CODE SPECIFIC TO HYPEROPT
#YOU WILL SEE THAT THERE IS CODE THAT USES PICKLE TO SAVE THE HYPERPARAMETERS SAVED BY HYPEROPT
#I EDITED THE CODE TO MAKE IT MORE BARE BONES. MIGHT WANT TO ADD LINE THAT SAVES MODEL

def optimize_cnn(space):
    """Build a convolutional neural network and train it for optimization."""
    
    # need to delete "model" so dict sequence can be updated
    
    try:
        model, results, x, y, z = train_model(space)
        
        print(f"\nHyperopt min: {results['loss']}\n")
        
        #COULD PUT CODE THAT SAVES CURRENT MODEL; THOUGH IT WILL NOT BE WITH HYPERPARAMETERS SELECTED BY
        #HYPEROPT

        K.clear_session()
        
        del model, x, y, z

        return results

    except Exception as err:
        try:
            K.clear_session()
        except:
            pass
        
        #this really isnt needed in jupyter notebook
        err_str = str(err)
        print(err_str)
        traceback_str = str(traceback.format_exc())
        print(traceback_str)
        return {
            'status': STATUS_FAIL,
            'err': err_str,
            'traceback': traceback_str
        }

    print("\n\n")


def run_trial(space, save_model_str):
    """Run one TPE meta optimization step and save its results."""
    max_evals = nb_evals = 1
 

    print("Attempt to resume a past training if it exists:")
    os.chdir('/home/mikedoho/Desktop/utsw_hn_ai/opo_hn_LN_prediction/Model_Trial_Results')

    try:
        # https://github.com/hyperopt/hyperopt/issues/267
        
        print('\nEvaluating using hyperparamers from {}_hype_space'.format(space['which_model']))
        print('Loading {} Model from '.format(space['which_model'].capitalize()), os.getcwd())
        trials = pickle.load(open(f"results_{space['which_model']}_{save_model_str}.pkl", 'rb'))
        print('Model loaded!\n')
        
        #print("Found saved Trials! Loading...")
        max_evals = len(trials.trials) + nb_evals
        print("Rerunning from {} trials to add another one.".format(
            len(trials.trials)))

    
    except:
        trials = Trials()
        print("Starting from scratch: new trials.")
    
    
        
    best = fmin(
                fn=optimize_cnn,
                space=space,
                algo=tpe.suggest,
                trials=trials,
                max_evals=max_evals
    )
    
    #Save Parameter space for Cross Validations
#     os.chdir('/home/mikedoho/Desktop/utsw_hn_ai/opo_hn_LN_prediction/Parameter_Space_CV')
#     with open(f"Parameter_space_CV", 'wb') as f:
#         pickle.dump(trials.trials[-1]['result']['space'], f)
    
    os.chdir(f"/home/mikedoho/Desktop/utsw_hn_ai/opo_hn_LN_prediction/Models/Best_Hyperopt_Models/{space['which_model']}")
    
    with open(f"Best_{space['which_model']}_Model_{save_model_str}", 'wb') as f:
        pickle.dump(space_eval(space, best), f)
    
    #This is a line of code that will need to change if reused
    os.chdir('/home/mikedoho/Desktop/utsw_hn_ai/opo_hn_LN_prediction/Model_Trial_Results')
    
    print('Saving {} Model to '.format(space['which_model'].capitalize()), os.getcwd())
    trials = pickle.dump(trials, open(f"results_{space['which_model']}_{save_model_str}.pkl", 'wb'))
    print('Model saved!')
    
    print("\nOPTIMIZATION STEP COMPLETE.\n")
    print('\nCurrent best:\n',space_eval(space, best),'\n')
   
    return best, space_eval(space, best)

In [ ]:
#Saving Model
save_model_str='postmanu_3sen_spec_1'
space = unet_hype_space

while True:
    

    try:
        best, num_space = run_trial(space=space, 
                                save_model_str=save_model_str)
        
    except: pass